<a href="https://colab.research.google.com/github/mhkarimid/Bioox-/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.5"
export JULIA_PKG_PRECOMPILE_AUTO=0
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  echo "Installing PyCall.jl..."
  julia -e 'using Pkg; Pkg.add("PyCall"); Pkg.build("PyCall")'
  julia -e 'println("Success")'

fi

Installing Julia 1.8.5 on the current Colab Runtime...
2023-03-18 01:25:39 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.5-linux-x86_64.tar.gz [130873886/130873886] -> "/tmp/julia.tar.gz" [1]
Installing PyCall.jl...
  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Preferences ───── v1.3.0
   Installed VersionParsing ── v1.3.0
   Installed Conda ─────────── v1.8.0
   Installed JSON ──────────── v0.21.3
   Installed MacroTools ────── v0.5.10
   Installed Parsers ───────── v2.5.8
   Installed PyCall ────────── v1.95.1
   Installed SnoopPrecompile ─ v1.0.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [438e738f] + PyCall v1.95.1
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8f4d0f93] + Conda v1.8.0
  [682c06a0] + JSON v0.21.3
  [1914dd2f] + MacroTools v0.5.10
  [69de0a69] + Parsers v2.5.8
  [21216c6a] + Preferences v1.3.0
  [438e738f] + 

In [3]:
%pip install -Uq pysr pytorch_lightning
from julia import Julia

julia = Julia(compiled_modules=False, threads='auto')
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()
import pysr

# We don't precompile in colab because compiled modules are incompatible static Python libraries:
pysr.install(precompile=False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.8 MB/s eta 0:00:00
Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      Ubuntu 20.04.5 LTS
  uname: Linux 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz: 
              speed         user         nice          sys         idle          irq
       #1  2200 MHz    

[ Info: Julia version info
[ Info: Julia executable: /usr/local/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /usr/bin/python3
│     libpython: /usr/lib/x86_64-linux-gnu/libpython3.9.so.1.0
│ Python:
│     python: /usr/bin/python3
└     libpython: 
     Cloning git-repo `https://github.com/MilesCranmer/SymbolicRegression.jl`
     Cloning git-repo `https://github.com/JuliaParallel/ClusterManagers.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed RealDot ────────────────────────── v0.1.0
   Installed IRTools ────────────────────────── v0.4.8
   Installed BitTwiddlingConvenienceFunctions ─ v0.1.5
   Installed DynamicPolynomials ─────────────── v0.4.6
   Installed GPUArraysCore ──────────────────── v0.1.4
   Installed SIMDTypes ──────────────────────── v0.1.0
   Installed IrrationalConstants ────────────── v0.2.2
   Installed AutoH

In [4]:
import sympy
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [5]:
df=pd.read_csv('https://raw.githubusercontent.com/mhkarimid/Data/main/newdata.csv', 
               names=['Time (min)', 'Gold grade (g/ton)', 'Pyrite content (wt.%)',
               'Ore content (wt.%)', 'pH', 'ORP (mV)', 'D-Gal (wt.%)',
               'D-Suc (wt.%)', 'D-FrU (wt.%)', 'Ferric (g/lit)', 'Pyrite dissolution (%)'])
df.head()

,Time (min),Gold grade (g/ton),Pyrite content (wt.%),Ore content (wt.%),pH,ORP (mV),D-Gal (wt.%),D-Suc (wt.%),D-FrU (wt.%),Ferric (g/lit),Pyrite dissolution (%)
0,-0.500000,0.5,0.5,-0.444444,-0.191734,-0.460602,-0.155844,-0.155844,-0.155844,-0.177262,-0.339150
1,-0.456522,0.5,0.5,-0.444444,-0.201303,-0.443605,-0.155844,-0.155844,-0.155844,-0.161934,-0.323117
2,-0.413043,0.5,0.5,-0.444444,-0.244366,-0.432274,-0.155844,-0.155844,-0.155844,-0.165186,-0.327591
3,-0.369565,0.5,0.5,-0.444444,-0.273074,-0.304795,-0.155844,-0.155844,-0.155844,-0.131744,-0.278188
4,-0.326087,0.5,0.5,-0.444444,-0.225227,-0.214143,-0.155844,-0.155844,-0.155844,-0.118739,-0.251342


In [6]:

gg=np.sqrt(df['Time (min)']**2+
            df['Gold grade (g/ton)']**2+
            df['Pyrite content (wt.%)']**2+
            df['Ore content (wt.%)']**2+
            df['pH']**2+\
            df['ORP (mV)']**2+
            df['D-Gal (wt.%)']**2+
            df['D-Suc (wt.%)']**2+
            df['D-FrU (wt.%)']**2)
    
df['Time (min)']=df['Time (min)']/gg
df['Gold grade (g/ton)']=df['Gold grade (g/ton)']/gg
df['Pyrite content (wt.%)']=df['Pyrite content (wt.%)']/gg
df['Ore content (wt.%)']=df['Ore content (wt.%)']/gg
df['pH']=df['pH']/gg
df['ORP (mV)']=df['ORP (mV)']/gg
df['D-Gal (wt.%)']=df['D-Gal (wt.%)']/gg
df['D-Suc (wt.%)']=df['D-Suc (wt.%)']/gg
df['D-FrU (wt.%)']=df['D-FrU (wt.%)']/gg

def sig(x):
 return 1/(1 + np.exp(-x))
 

df=sig(df)
df.head()

,Time (min),Gold grade (g/ton),Pyrite content (wt.%),Ore content (wt.%),pH,ORP (mV),D-Gal (wt.%),D-Suc (wt.%),D-FrU (wt.%),Ferric (g/lit),Pyrite dissolution (%)
0,0.390836,0.609164,0.609164,0.402637,0.457557,0.399193,0.465473,0.465473,0.465473,0.455800,0.416016
1,0.397959,0.611448,0.611448,0.400586,0.454491,0.400769,0.464729,0.464729,0.464729,0.459605,0.419916
2,0.406363,0.612734,0.612734,0.399431,0.444174,0.402113,0.464309,0.464309,0.464309,0.458797,0.418827
3,0.411799,0.618307,0.618307,0.394420,0.434517,0.427013,0.462484,0.462484,0.462484,0.467112,0.430898
4,0.418112,0.624058,0.624058,0.389243,0.443173,0.445947,0.460590,0.460590,0.460590,0.470350,0.437493


In [11]:
XX=df.drop(['Gold grade (g/ton)','Ferric (g/lit)','Pyrite dissolution (%)'], axis=1)
yy=df['Pyrite dissolution (%)']

Rtr=[]
RMSEtr=[]

Rte=[]
RMSEte=[]

MODEL=[]

for i in range(1,19):
    X_tr, X_te, y_tr, y_te = train_test_split(XX, yy, test_size=0.05*i)
    X = X_tr.values.tolist()
    y = y_tr.transpose().values.tolist()
    Xte = X_te.values.tolist()
    yte = y_te.transpose().values.tolist()
    default_pysr_params = dict(
        populations=10,
        model_selection="accuracy",
        loss="L1DistLoss()",
        maxsize=30,
        precision=16,
    )
    model = PySRRegressor( 
        niterations=30,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["exp","sinh", "cosh"],
        **default_pysr_params
    )
    model.fit(X, y)

    MODEL.append(model)
    y2=model.predict(Xte)
    
    dk=pd.DataFrame(data=[yte,y2])
    d=dk.transpose().corr()[1]
    SS=d.squeeze()[0]
    Rte.append(SS)
    rmse=(mean_squared_error(yte, y2))
    RMSEte.append(rmse)  
    
    y3=model.predict(X)
    dk=pd.DataFrame(data=[y,y3])
    d=dk.transpose().corr()[1]
    SS=d.squeeze()[0]
    Rtr.append(SS)
    rmse=(mean_squared_error(y, y3))
    RMSEtr.append(rmse)
    
    


/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 2.870e+02
Head worker occupation: 0.0%
Progress: 1 / 300 total iterations (0.333%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.524e-02  7.311e-01  x4

Press 'q' and then <enter> to stop execution early.

Cycles per second: 1.820e+03
Head worker occupation: 0.3%
Progress: 11 / 300 total iterations (3.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.524e-02  7.311e-01  x4
3           2.502e-02  4.168e-03  (x4 / 1.004)
10          2.457e-02  2.605e-03  (x4 * exp((sinh(-0.3743) * x4) * sinh(0.07324)))
12          2.293e-02  3.436e-02  (x4 * exp((-0.3743 * x4) * ((x4 + -0.3743) - 0.07324)))
13          2.290e-02  1.466e-03  (x4 * exp((sinh(-0.3743) * x4) * ((x4 + -0.3743) - 0.07324)))
14          2.238e-02  2.272e-02  (x4 * exp(((-0.3743 / x1) * x4) * ((x4 + -0.3743) - 0.07324)))
18          2.228e-02  1.224e-03  (x4 * exp(((-0

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 1.060e+03
Head worker occupation: 0.1%
Progress: 3 / 300 total iterations (1.000%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.522e-02  7.375e-01  x4
11          2.472e-02  2.022e-03  (((0.3716 / cosh(sinh(x4) + x2)) + 0.7803) * x4)
13          2.220e-02  5.382e-02  (((0.3716 / cosh(x4 + (x4 / x1))) + 0.7803) * sinh(x4))
18          2.203e-02  1.470e-03  ((sinh(sinh(0.3716) / ((x0 + sinh(sinh(x4) + x4)) + 0.3716)) + 0.7695) * sinh(x4))
21          2.132e-02  1.109e-02  (((sinh(0.3716) / ((x4 + sinh((sinh(x4) + (x4 - x1)) + x4)) + x1)) + 0.7695) * sinh(x4))
22          2.074e-02  2.772e-02  (((sinh(sinh(0.3716)) / ((x4 + sinh((sinh(x4) + (x4 - x1)) + x4)) + 0.4158)) + 0.7695) * sinh(x4))
24          1.971e-02  2.529e-02  (((sinh(sinh(0.3716)) / ((x4 + sinh((sinh(x4) + ((x4 - x1) / x1)) + x4)) + x1)) + 0.7695) * sinh(x4))

Press 'q' and then <enter> to stop execution early.

Cycles per second

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


...

Started!

Cycles per second: 7.880e+03
Head worker occupation: 1.4%
Progress: 23 / 300 total iterations (7.667%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.415e-02  7.544e-01  x4
3           2.379e-02  7.626e-03  (x4 / 1.009)
5           2.016e-02  8.293e-02  ((x4 * 0.669) + 0.1628)
11          1.993e-02  1.882e-03  (x4 + (-0.03757 * ((x4 - 0.4827) / (x3 * x6))))
13          1.894e-02  2.554e-02  ((x4 * (0.2429 + (sinh(x4) * (0.2429 + 0.2429)))) + sinh(0.2429))
23          1.878e-02  8.335e-04  (x4 + (-0.03757 * ((sinh(((x4 - (sinh(x7) / (x4 / sinh(0.6396)))) * 0.958) / x7) + x2) * cosh(x0))))
27          1.871e-02  9.785e-04  (x4 + (-0.03757 * sinh(((sinh(x4 - (x6 / (x4 / 0.6396))) / (sinh(x6) / ((0.6396 / sinh(0.6396)) / x4))) + x2) * x3)))

Press 'q' and then <enter> to stop execution early.

Cycles per second: 8.890e+03
Head worker occupation: 1.0%
Progress: 51 / 300 total iterations (17.000%)
Hall of Fame:
------

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 8.910e+03
Head worker occupation: 1.1%
Progress: 25 / 300 total iterations (8.333%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.446e-02  7.648e-01  x4
3           2.406e-02  8.122e-03  (x4 * 0.991)
5           2.173e-02  5.111e-02  sinh(exp(-0.3726 / x4))
11          2.034e-02  1.102e-02  sinh(exp(sinh(sinh(x4) * sinh(x4)) - cosh(0.257)))
12          1.961e-02  3.680e-02  sinh(sinh(exp(sinh(x4 * x4) - cosh(0.1406 / x4))))
14          1.945e-02  3.922e-03  exp((sinh(sinh(x4)) * sinh(sinh(x4))) - cosh(sinh(x7) - x4))
15          1.901e-02  2.322e-02  sinh(exp((sinh(x4) * sinh(x4)) - cosh((x7 * sinh(x4)) - x7)))
16          1.889e-02  6.368e-03  sinh(exp(sinh(sinh(x4) * sinh(x4)) - cosh((x7 * sinh(x4)) - x7)))
18          1.851e-02  1.011e-02  sinh(exp((sinh(x4) * sinh(x4)) - cosh(((x7 * x4) * (x1 / x2)) - x7)))
19          1.834e-02  9.321e-03  sinh(exp(sinh(sinh(x4) * sinh(x4)) - cosh(((x7 *

/usr/local/lib/python3.9/dist-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pysr/julia_helpers.py:217: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(


Started!

Cycles per second: 8.770e+03
Head worker occupation: 1.2%
Progress: 25 / 300 total iterations (8.333%)
Hall of Fame:
-----------------------------------------
Complexity  Loss       Score     Equation
1           2.484e-02  7.417e-01  x4
3           2.390e-02  1.942e-02  (x4 * 0.984)
6           2.356e-02  4.754e-03  (x4 / cosh(0.0781 / x1))
8           2.155e-02  4.466e-02  sinh(((x4 * 0.828) * x4) - -0.2664)
15          2.095e-02  4.032e-03  (((((-0.03223 * -0.4587) - -0.4587) * 1.079) * sinh(sinh(sinh(x4)))) - sinh(-0.2036))
16          2.028e-02  3.276e-02  (((((sinh(x3) * -0.03223) - -0.4587) * 1.079) * sinh(sinh(sinh(sinh(x4))))) - -0.2036)
22          2.019e-02  7.340e-04  sinh(((((sinh(((x1 * x4) * sinh(x4)) * 0.1348) - -0.2096) - -0.2112) * 1.093) * sinh(sinh(x4))) - -0.2096)
26          1.970e-02  6.179e-03  (((((sinh(((sinh(sinh(x1)) + -0.4587) * 1.079) * sinh(cosh(sinh(x6)))) * 0.05423) - -0.4587) * 1.079) * sinh(sinh(sinh(x4)))) - -0.2051)
29          1.968e-02  

In [8]:
Results=pd.DataFrame(list(zip(RMSEtr, Rtr, RMSEte, Rte)), columns=['RMSE (training)', 
                                                               'Correlation factor (training)',
                                                              'RMSE (testing)', 
                                                               'Correlation factor (testing)'])

In [9]:
Results

,RMSE (training),Correlation factor (training),RMSE (testing),Correlation factor (testing)
0,0.000491,0.929603,0.000534,0.921149
1,0.000403,0.941890,0.000432,0.940572
2,0.000437,0.937308,0.000446,0.932700
3,0.000439,0.936744,0.000433,0.936749
4,0.000510,0.926749,0.000484,0.928408
5,0.000495,0.931612,0.000533,0.923495
6,0.000482,0.931744,0.000491,0.932869
7,0.000403,0.940640,0.000433,0.940472
8,0.000406,0.942780,0.000427,0.937425
9,0.000508,0.928134,0.000481,0.934123


In [12]:
Results.to_excel("SRGAResutls.xlsx") 

In [13]:
MODEL

[PySRRegressor.equations_ = [
 	    pick     score                                           equation  \
 	0         0.000000                                                 x4   
 	1         0.007384                                        (x4 / 1.01)   
 	2         0.079359                           ((x4 + 0.2012) * 0.7095)   
 	3         0.009755                 sinh(sinh((x4 - -0.2278) * 0.632))   
 	4         0.011074      ((1.049 - ((0.457 - x4) * exp(x6))) * 0.4424)   
 	5         0.010989  ((1.049 - ((0.457 - x4) * exp(sinh(x6)))) * 0....   
 	6         0.006045  ((1.053 - ((0.4556 - x4) * cosh(x7 / x3))) * 0...   
 	7         0.015275  ((1.053 - ((0.4714 - sinh(x4)) * cosh(x7 + x4)...   
 	8         0.013430  ((1.049 - ((0.457 - x4) * exp((x4 + x1) * x7))...   
 	9         0.022083  sinh((1.007 - (((0.489 - x4) * x4) * cosh(exp(...   
 	10        0.045685  sinh((1.007 - (((0.489 - x4) * x4) * (sinh(exp...   
 	11        0.013446  sinh((1.007 - (((0.489 - x4) * x4) * (sinh(exp..

In [16]:
for i in range(17):
  print(MODEL[i].sympy())

-0.4585*x4*(0.489 - sinh(x4))*(x6 + sinh(sinh(x5 + sinh(-x3 + x6 + x7))))*cosh(x1 + x4) + 0.4617095
x4 + 0.696421500026267*(0.4526 - x4)/(x4**2*(x4 + 2.512)*cosh(sinh(cosh(sinh(1.62074554294976*x6*sinh(x7)/x3)))))
x4 - 0.010796*sinh((x4 - 0.552/x4)/x5) - 0.010796*sinh(sinh(x2) + sinh(0.485672656629432*cosh(1.11111111111111*sinh(x4)/sinh(x1))) + 0.02116)
-x3*(x4 - 0.4497)*sinh((sinh(x4) - 0.339/x6)/((0.01782 - cosh(x4 + sinh(x7)))*sinh(sinh(x5))))/x7 + x4
sinh(x4 - 0.0962*(x2 + 0.4702)*(x4 - 0.4192)/(x1*sinh(x0*x4)*sinh(x6*(x6 + cosh(exp(x7))))))
x4 - 1.431*sinh(sinh(x4) - 0.469984340809377)/(-x3 + sinh(x5*(x1 + sinh(cosh(x4 + x6 + sinh(sinh(x7)))))))
x3*(sinh(sinh(sinh(x4))) - 0.51)*(x6 + sinh(x7 - 0.252) - 0.865)*sinh((-x4 + cosh(x5 - cosh(x4)))/x4) + x4
sinh(0.332*cosh(1.12994350282486*x4 + 1.12994350282486*sinh(0.268744961031981*exp(x7*(x6 + (cosh(x1) - 0.637/x4)*exp(x5)*exp(x6))))))
0.421 - 0.2015*sinh(x6*x7 - (x4*sinh(x4) - 0.0919)*(x2 + 4.605*x6*x7*exp(x1) + x6/x2))
sinh(sinh(sin

In [17]:
MODEL[0].sympy()

-0.4585*x4*(0.489 - sinh(x4))*(x6 + sinh(sinh(x5 + sinh(-x3 + x6 + x7))))*cosh(x1 + x4) + 0.4617095

In [18]:
MODEL[1].sympy()

x4 + 0.696421500026267*(0.4526 - x4)/(x4**2*(x4 + 2.512)*cosh(sinh(cosh(sinh(1.62074554294976*x6*sinh(x7)/x3)))))

In [19]:
MODEL[2].sympy()

x4 - 0.010796*sinh((x4 - 0.552/x4)/x5) - 0.010796*sinh(sinh(x2) + sinh(0.485672656629432*cosh(1.11111111111111*sinh(x4)/sinh(x1))) + 0.02116)

In [20]:
MODEL[3].sympy()

-x3*(x4 - 0.4497)*sinh((sinh(x4) - 0.339/x6)/((0.01782 - cosh(x4 + sinh(x7)))*sinh(sinh(x5))))/x7 + x4

In [21]:
MODEL[4].sympy()

sinh(x4 - 0.0962*(x2 + 0.4702)*(x4 - 0.4192)/(x1*sinh(x0*x4)*sinh(x6*(x6 + cosh(exp(x7))))))

In [22]:
MODEL[5].sympy()

x4 - 1.431*sinh(sinh(x4) - 0.469984340809377)/(-x3 + sinh(x5*(x1 + sinh(cosh(x4 + x6 + sinh(sinh(x7)))))))

In [23]:
MODEL[6].sympy()

x3*(sinh(sinh(sinh(x4))) - 0.51)*(x6 + sinh(x7 - 0.252) - 0.865)*sinh((-x4 + cosh(x5 - cosh(x4)))/x4) + x4

In [24]:
MODEL[7].sympy()

sinh(0.332*cosh(1.12994350282486*x4 + 1.12994350282486*sinh(0.268744961031981*exp(x7*(x6 + (cosh(x1) - 0.637/x4)*exp(x5)*exp(x6))))))

In [25]:
MODEL[8].sympy()

0.421 - 0.2015*sinh(x6*x7 - (x4*sinh(x4) - 0.0919)*(x2 + 4.605*x6*x7*exp(x1) + x6/x2))

In [26]:
MODEL[9].sympy()

sinh(sinh(sinh(x4 + (-0.01456*sinh(x4) - 0.0203336681704448*(x6 + (x4 - 0.50036)/(x4*x7))/x1)/x6)))

In [27]:
MODEL[10].sympy()

sinh(sinh(sinh(0.6157*x4**4*(x1*x5*x7*sinh(cosh(sinh(cosh(sinh(x6))))) + 1.425) + 0.363)))

In [28]:
MODEL[11].sympy()

0.02548*x0 - 0.01274*x2 + x4 - 0.01274*sinh(sinh(x2)) + 0.01274*cosh(0.3513/sinh(x4)**2) - 0.01274*cosh(cosh(x4/x1))

In [29]:
MODEL[12].sympy()

0.3015*x3*(0.298219584569733*x4 - 0.1333836)*sinh(x4 - 0.6733)/(x6**2*x7*sinh(x5)*sinh(x7)) + x4

In [30]:
MODEL[13].sympy()

sinh(sinh(sinh(x4 - 0.11002 + 0.02374/((x0*x5 + sinh(sinh(x4)))*sinh(sinh(sinh(sinh(sinh(sinh(sinh(x4)))))) - 0.102538841002623)))))

In [31]:
MODEL[14].sympy()

x5 - (-x4 + x5)*sinh(x5 - x7*sinh(sinh((x3 - x4)*cosh(sinh((sinh(x5) - 0.3596/x4)/x0)) - sinh(x6))))

In [32]:
MODEL[15].sympy()

0.357721688980454*exp(sinh(x4*sinh(sinh(x0*x4*(x4 + x6)*(x4 + x6 - 0.3318/x1)*sinh(sinh(x7)) + x4))))

In [33]:
MODEL[16].sympy()

(1.06279262961009*x4*x6*exp(-sinh(0.1862/(x1*x4*(x0 + x4*(x4 - 0.3286)/x2)))) + 0.3193)*sinh(x4) + 0.286

In [34]:
MODEL[17].sympy()

(0.1473364*x0*x6*(x4 + sinh((x0 + x7)*sinh(x1)/x2) - 0.2961) + 0.4581856)*sinh(sinh(x4)) + 0.2096

In [36]:
X_tr, X_te, y_tr, y_te = train_test_split(XX, yy, test_size=0.05*2)
X = X_tr.values.tolist()
y = y_tr.transpose().values.tolist()
Xte = X_te.values.tolist()
yte = y_te.transpose().values.tolist()

ytr=MODEL[1].predict(X)
yte=MODEL[1].predict(Xte)